# Lemmatization reserach
In this file you can see codes I created to asess which package is better to use for lemmatization of a db
There are two packages that I use: STANZA and Spacy
At the end in result of manual check by referees it was decided to not use customized pipilines and to use Spacy instead os `stanza

In [ ]:
import pickle
import pandas as pd
import numpy as np
import spacy
import re
import stanza
import sqlite3
import time
import string
from spacy.cli import download
from spacy.lang.it.stop_words import STOP_WORDS

In [7]:
stanza.download('it')
it_nlp_stanza = stanza.Pipeline('it', processors='tokenize,mwt,pos,lemma', verbose=False, use_gpu=False)

try:
    it_nlp_spacy = spacy.load('it_core_news_lg')
except OSError:
    download('it_core_news_lg')
    it_nlp_spacy = spacy.load('it_core_news_lg')



In [8]:
conn = sqlite3.connect('/g100_work/IscrC_mental/data/database/MENTALISM.db')

# Defining function for downloading, lemmatization, cleaning

In [43]:
def download_batch(batch_size):
    start_time = time.time()
    your_query = f"SELECT tweet_id, text, language FROM tweets limit {batch_size}"
    df = pd.read_sql_query(your_query, conn)
    elapsed_time = time.time() - start_time
    print(f"\nTime taken to download batch of size {batch_size}: {elapsed_time:.2f} seconds")
    return df


def itallian_df(df):
    start_time = time.time()
    itallian_df = df[df['language'] == 'it']
    print(f"\nTime taken for 'itallian_df': {time.time() - start_time:.2f} seconds")
    print(f"\nLength of downloaded df {len(df)}")
    return itallian_df

def cleaning_df(df):
    start_time = time.time()
    def cleaning(text):
        
        stop_words = STOP_WORDS
        stop_words_to_remove = ['anni', 'anno']
        stop_words = [word for word in stop_words if word not in stop_words_to_remove]
        if pd.isna(text):
                return ""
        words_new = [word[1:].translate(str.maketrans('', '', string.punctuation)) 
        if (word.startswith('#') or word.startswith('@')) else word.lower().translate(str.maketrans('', '', string.punctuation))
        for word in text.split()
        if not (re.match(r"http\S+|www\S+|https\S+", word) or word.lower() in stop_words)]
        filtered_text = ' '.join(words_new)
        return filtered_text

    df['text'] = df['text'].apply(cleaning)
    print(f"\nTime taken for 'cleaning_df': {time.time() - start_time:.2f} seconds")
    return df


def lemmatizationSpacy_df(df, nlp_model=it_nlp_spacy):
    start_time = time.time()
    def lemmatizationSpacy(text):
        tokens = nlp_model(text)
        lemmatized_tweet = " ".join([token.lemma_ for token in tokens])
        return lemmatized_tweet
    df['text'] = df['text'].apply(lemmatizationSpacy)
    print(f"\nTime taken for 'spacy': {time.time() - start_time:.2f} seconds")
    return df


def lemmatizationStanza_df(df):
    start_time = time.time()
    def lemmatizationStanza(text):
        doc = it_nlp_stanza(text)
        lemmatized_text = ' '.join([word.lemma if word.lemma.endswith((',', '.')) else word.lemma + ' ' for sent in doc.sentences for word in sent.words])
        return lemmatized_text
    df['text'] = df['text'].apply(lemmatizationStanza)
    print(f"\nTime taken for 'Stanza': {time.time() - start_time:.2f} seconds")
    return(df)


def save_to_csv(df, output_file='output_file.csv'):
    start_time = time.time()
    df.to_csv(output_file, index=False)
    print(f"\nDataFrame saved to {output_file}")
    print(f"\nTime taken for 'saving': {time.time() - start_time:.2f} seconds")


In [72]:
batch = download_batch(10000)
ital = itallian_df(batch)


Time taken to download batch of size 10000: 0.48 seconds


In [73]:
ital = df.drop('language', axis =1) 


Time taken for 'itallian_df': 0.23 seconds

Length of downloaded df 10000


In [25]:
df = lemmatizationStanza_df(df)


Time taken for 'Stanza': 701.40 seconds


/scratch_local/slurm_job.11639884/ipykernel_7762/2199210308.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lemmatizationStanza)


# Building pipilines components for spacy

In [6]:
nlp = spacy.load('it_core_news_lg')

### for future how to display whole tweets 
pd.set_option('display.max_colwidth', None)

components_to_remove = ['tagger', 'parser', 'attribute_ruler', 'ner']
for component in components_to_remove:
    nlp.remove_pipe(component)

In [10]:
### adding cleaning component

from spacy.language import Language

@Language.component("cleaning")
def cleaning(doc):
    stop_words = spacy.lang.it.stop_words.STOP_WORDS
    new_tokens = [token.text for token in doc if not re.match(r"http\S+|www\S+|https\S+|@\S+|#(?!\w)", token.text) and token.text.lower() not in stop_words]
    return spacy.tokens.Doc(doc.vocab, words=new_tokens)

nlp.add_pipe("cleaning", name="cleaning", first=True)
print(nlp.pipe_names)

lemmatized_tweets_spacy = []
for tweet in tweets:
    tokens = nlp(tweet)
    lemmatized_tweet = " ".join([token.lemma_ for token in tokens])
    lemmatized_tweets_spacy.append(lemmatized_tweet)

In [14]:
import stanza
stanza.download('it')

2023-11-13 15:32:08 INFO: Downloading default packages for language: it (Italian) ...
2023-11-13 15:32:09 INFO: File exists: /g100/home/userexternal/ddurmush/stanza_resources/it/default.zip
2023-11-13 15:32:13 INFO: Finished downloading models and saved to /g100/home/userexternal/ddurmush/stanza_resources.


In [15]:
it_nlp = stanza.Pipeline('it', processors='tokenize,mwt,pos,lemma', verbose=False, use_gpu=False)

In [16]:
### lets just clean it here 
## stanza does not have stop words for italian

tweets_no_links_mentions = [re.sub(r"(http\S+|www\S+|https\S+|@\S+)", "", tweet, flags=re.MULTILINE) for tweet in tweets]
stop_words = spacy.lang.it.stop_words.STOP_WORDS

tweets_no_links_mentions_stopwords = []

for tweet in tweets_no_links_mentions:
    filtered_tweet = ' '.join(word for word in tweet.split() if word.lower() not in stop_words)
    tweets_no_links_mentions_stopwords.append(filtered_tweet)
    
tweets_no_hashtags = [
    re.sub(r'#', '', tweet)
    for tweet in tweets_no_links_mentions_stopwords
]

In [ ]:
### adding a costumised stage of pipiline for deleting #,@ and stopwords

class RemoveTweetsMentionsStopwordsProcessor:
    def __init__(self, device, config, pipeline):
        self.stop_words = {'stopword1', 'stopword2', 'stopword3'}  

    def _set_up_model(self, *args):
        pass

    def process(self, doc):
        tweets_mentions_stopwords_removed = []

        for sent in doc.sentences:
            cleaned_tokens = []
            for tok in sent.tokens:

                if (
                    not tok.text.startswith('@')
                    or not tok.text.startswith('#')
                    or tok.text.lower() not in self.stop_words
                ):
                    cleaned_tokens.append(tok.text)

            cleaned_sentence = ' '.join(cleaned_tokens)
            tweets_mentions_stopwords_removed.append(cleaned_sentence)

        doc.text = ' '.join(tweets_mentions_stopwords_removed)

        return doc

stanza.download('it')

custom_pipeline = stanza.Pipeline('it', processors={'tokenize': 'it', 'remove_tweets_mentions_stopwords': RemoveTweetsMentionsStopwordsProcessor})

text = "This is a @mention and #hashtag example."
doc = custom_pipeline(text)

cleaned_text = doc.text
print("Cleaned Text:", cleaned_text)

In [ ]:
from stanza.pipeline.processor import ProcessorVariant, register_processor_variant
from stanza.pipeline.processor import Processor
from stanza.pipeline.core import register_processor

@register_processor("remove_tweets_mentions")
class RemoveTweetsMentionsProcessor(Processor):
    ''' Processor that cleanes '''
    _requires = set(['tokenize'])
    _provides = set(['cleaned'])

    def __init__(self, device, config, pipeline):
        pass

    def _set_up_model(self, *args):
        pass

    def process(self, doc):
        tweets_mentions_removed = []

        for sent in doc.sentences:
            cleaned_tokens = []
            for tok in sent.tokens:
                if (not tok.text.startswith('@')
                    or not tok.text.startswith('#')
                    or not re.match(r"http\S+|www\S+|https\S+|@\S+|#(?!\w)", tok.text)
                    or tok.text.lower() not in stop_words):
                    cleaned_tokens.append(tok.text)

            cleaned_sentence = ' '.join(cleaned_tokens)
            tweets_mentions_removed.append(cleaned_sentence)

        doc.text = ' '.join(tweets_mentions_removed)
        
        return doc

In [ ]:
@register_processor("cleaning")
class LowercaseProcessor(Processor):
    ''' Processor that lowercases all text '''
    _requires = set(['tokenize'])
    _provides = set(['lowercase'])

    def __init__(self, device, config, pipeline):
        pass

    def _set_up_model(self, *args):
        pass

    def process(self, doc):
        doc.text = doc.text.lower()
        for sent in doc.sentences:
            for tok in sent.tokens:
                tok.text = tok.text.lower()

            for word in sent.words:
                word.text = word.text.lower()
                
        return doc

In [ ]:
nlp = stanza.Pipeline(dir=TEST_MODELS_DIR, lang='en', processors='tokenize,lowercase')

In [17]:
### pipeline with tdqm because it takes a lot of time to process 

import time
from tqdm.notebook import tqdm

lemmatized_tweets_stanza = []
total_start_time = time.time() 

progress_bar = tqdm(total=len(tweets), desc="Processing tweets", unit="tweet", leave=False)

for tweet in tweets_no_hashtags:
    doc = it_nlp(tweet)
    lemmatized_tweet = ' '.join([word.lemma if word.lemma.endswith((',', '.')) else word.lemma + ' ' for sent in doc.sentences for word in sent.words])
    lemmatized_tweets_stanza.append(lemmatized_tweet)
    

    progress_bar.update(1)


progress_bar.close()

total_end_time = time.time()
total_time = total_end_time - total_start_time
average_time = total_time / len(tweets) if len(tweets) > 0 else 0

print(f"\nTotal time for all iterations: {total_time:.2f} seconds")
print(f"Average time per item: {average_time:.4f} seconds")



Processing tweets:   0%|          | 0/1119 [00:00<?, ?tweet/s]


Total time for all iterations: 167.74 seconds
Average time per item: 0.1499 seconds


In [19]:
df_to_check = pd.DataFrame({'original': tweets, 'lemmatized_spacy':lemmatized_tweets_spacy, 'lemmatized_stanza':lemmatized_tweets_stanza  })
df_to_check.to_csv('lemmas_spacy_stanza.csv')